In [1]:
import numpy as np
import itertools
from scipy.sparse.linalg import LinearOperator
from scipy import sparse

In [ ]:
dom = (2,3,4)
d = len(dom)
X = np.ones(dom)
tables= 2**d
ns = sorted(range(tables), key=lambda n: bin(n).count('1'), reverse=True)

marginals = [None]*tables
marginals[tables-1] = X

for cube in itertools.product(*[[0,1]]*d):
    axes = [i for i in range(d) if cube[i] == 0]
    print X.sum(axis=tuple(axes), keepdims=True)


In [ ]:
ns = sorted(range(tables), key=lambda n: bin(n).count('1'), reverse=True)
print ns

In [2]:
def marginals_linop(domain, weights):
    d = len(domain)
    tables = 2**d
    cubes = list(itertools.product(*[[1,0]]*d))
    lookup = [None]*tables
    wgts = [None]*tables
    wgts[-1] = weights[0]
    for wgt, cube in zip(weights, cubes[1:]):
        a = ''.join(map(str, cube))
        i = a.index('0')
        a = int(a,2)
        lookup[a] = (a + 2**(d-i-1), i)
        wgts[a] = wgt
    keys = list(sorted(range(tables), key=lambda n: bin(n).count('1'), reverse=True))
    
    m = np.prod([n+1 for n in domain])
    n = np.prod(domain)
    marginals = [None]*tables
    marginals[tables-1] = np.zeros(domain)
    for k in keys[1:]:
        prev, dim = lookup[k]
        marginals[k] = marginals[prev].sum(axis=dim, keepdims=True)
    sizes = np.cumsum([M.size for M in marginals])[:-1]
    
    def matvec(x):
        X = x.reshape(domain)
        marginals[-1] = wgts[-1]*X
        for k in keys[1:]:
            prev, dim = lookup[k]
            marginals[prev].sum(axis=dim, keepdims=True, out=marginals[k])
        return np.concatenate([w*M.flatten() for w, M in zip(wgts, marginals)])
    def rmatvec(y):
        vecs = [v.reshape(M.shape) for v, M in zip(np.split(y, sizes), marginals)]
        X = np.zeros(domain)
        split = np.split(y, sizes)
        for w, v, M in zip(wgts, split, marginals):
            X += w*v.reshape(M.shape)
        return X.flatten()
    return LinearOperator(shape=(m,n), matvec=matvec, rmatvec=rmatvec)

In [3]:
def all_marginals(domain, weights):
    result = []
    cubes = list(itertools.product(*[[1,0]]*len(domain)))
    subs = [(np.ones(n, dtype=np.float32), sparse.eye(n, dtype=np.float32)) for n in domain]
    for wgt, cube in zip(weights, cubes):
        if wgt > 0:
            sub = reduce(sparse.kron, [sub[i] for sub, i in zip(subs, cube)])
            result.append(wgt * sub)
    return sparse.vstack(result)


In [19]:
X = np.random.rand(2*3)
wgts = np.ones(4) #np.random.rand(8)
L = marginals_linop((2,3), wgts)
M = all_marginals((2,3), wgts)
x = X.flatten()
#print L.dot(x)
#print M.dot(x)

print L.rmatvec(L.dot(x))
print M.T.dot(M.dot(x))

[ 6.22386718  5.22361515  6.69779674  7.18341378  6.32368076  7.60796212]
[ 6.22386718  5.22361515  6.69779674  7.18341378  6.32368076  7.60796212]


In [15]:
print L.dot(np.eye(2))
print L.H.dot(np.eye(3))
print M.T.toarray()

[[ 1.  1.]
 [ 1.  0.]
 [ 0.  1.]]
[[ 1.  1.  0.]
 [ 1.  0.  1.]]
[[ 1.  0.  1.]
 [ 0.  1.  1.]]


In [ ]:
def M(n): return np.vstack([np.ones(n), np.eye(n)])
W = reduce(np.kron, [M(2), M(3), M(4)])
y = W.dot(x)
W.T.dot(y)

In [ ]:
sorted(range(10))

In [ ]:
''.join(map(str, cube)).replace(0,1)